In [2]:
import optuna
import joblib
import numpy as np
import pandas as pd
import sys
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import HyperparamTuner, parse_jazzy_df

importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [3]:
model_identifiers = ["linear", "KRR", "GB", "RF", "ANN"]
isozymes = ["3A4", "RLM", "HLC"]
data_splits = ["train", "test"]
splitters = ["rand", "scaff"]
splitters = ["rand", "scaff", "time"]
rel_paths = {
    "3A4_source": r"project_resources/ChEMBL_3A4.csv",
    "3A4_sep": ";",
    "3A4": r"project_resources/3A4.csv",
    "3A4_train_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_train.csv",
    "3A4_test_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_test.csv",
    "3A4_train_rand": r"project_resources/base_splits/random/3A4_train.csv",
    "3A4_test_rand": r"project_resources/base_splits/random/3A4_test.csv",
    "3A4_train_time": r"project_resources/base_splits/time_split/3A4_train.csv",
    "3A4_test_time": r"project_resources/base_splits/time_split/3A4_test.csv",
    "3A4_jazzy_train_rand": r"project_resources/jazzy_splits/random/3A4_train.csv",
    "3A4_jazzy_test_rand": r"project_resources/jazzy_splits/random/3A4_test.csv",
    "3A4_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/3A4_train.csv",
    "3A4_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/3A4_test.csv",
    "3A4_jazzy_train_time": r"project_resources/jazzy_splits/time_split/3A4_train.csv",
    "3A4_jazzy_test_time": r"project_resources/jazzy_splits/time_split/3A4_test.csv",

    "RLM_source": r"project_resources/AID_1508591_datatable_all.csv",
    "RLM_sep": ",",
    "RLM": r"project_resources/RLM.csv",
    "RLM_train_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_train.csv",
    "RLM_test_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_test.csv",
    "RLM_train_rand": r"project_resources/base_splits/random/RLM_train.csv",
    "RLM_test_rand": r"project_resources/base_splits/random/RLM_test.csv",
    "RLM_train_time": r"project_resources/base_splits/time_split/RLM_train.csv",
    "RLM_test_time": r"project_resources/base_splits/time_split/RLM_test.csv",
    "RLM_jazzy_train_rand": r"project_resources/jazzy_splits/random/RLM_train.csv",
    "RLM_jazzy_test_rand": r"project_resources/jazzy_splits/random/RLM_test.csv",
    "RLM_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/RLM_train.csv",
    "RLM_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/RLM_test.csv",
    "RLM_jazzy_train_time": r"project_resources/jazzy_splits/time_split/RLM_train.csv",
    "RLM_jazzy_test_time": r"project_resources/jazzy_splits/time_split/RLM_test.csv",

    "HLC_source": r"project_resources/AID_1508603_datatable_all.csv",
    "HLC_sep": ",",
    "HLC": r"project_resources/HLC.csv",
    "HLC_train_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_train.csv",
    "HLC_test_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_test.csv",
    "HLC_train_rand": r"project_resources/base_splits/random/HLC_train.csv",
    "HLC_test_rand": r"project_resources/base_splits/random/HLC_test.csv",
    "HLC_train_time": r"project_resources/base_splits/time_split/HLC_train.csv",
    "HLC_test_time": r"project_resources/base_splits/time_split/HLC_test.csv",
    "HLC_jazzy_train_rand": r"project_resources/jazzy_splits/random/HLC_train.csv",
    "HLC_jazzy_test_rand": r"project_resources/jazzy_splits/random/HLC_test.csv",
    "HLC_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/HLC_train.csv",
    "HLC_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/HLC_test.csv",
    "HLC_jazzy_train_time": r"project_resources/jazzy_splits/time_split/HLC_train.csv",
    "HLC_jazzy_test_time": r"project_resources/jazzy_splits/time_split/HLC_test.csv"
}
# sampler - a method used to generate new sets of hyperparameters in each iteration of the optimization process
samplers = {
    'RandomSampler': optuna.samplers.RandomSampler,          # Sampler that selects hyperparameters randomly from the search space.
    'GridSampler': optuna.samplers.GridSampler,              # Sampler that performs a grid search over the hyperparameter space.
    'TPESampler': optuna.samplers.TPESampler,                # Sampler that uses a tree-structured Parzen estimator to model the objective function and sample new points from the search space.
    'CmaEsSampler': optuna.samplers.CmaEsSampler,            # Sampler that uses the Covariance Matrix Adaptation Evolution Strategy algorithm to efficiently search the hyperparameter space.
    'NSGAIISampler': optuna.samplers.NSGAIISampler,          # Multi-objective evolutionary algorithm that generates new samples using non-dominated sorting and crowding distance selection.
    'QMCSampler': optuna.samplers.QMCSampler,                # Quasi-Monte Carlo sampler that uses low-discrepancy sequences to sample the search space in a more efficient and evenly distributed way than random sampling.
    'BoTorchSampler': optuna.integration.BoTorchSampler,     # Sampler that leverages the BoTorch library for Bayesian optimization and can handle both continuous and categorical hyperparameters.
    'BruteForceSampler': optuna.samplers.BruteForceSampler,  # Sampler that exhaustively evaluates all possible combinations of hyperparameters in the search space.
}
# pruner - a technique used to eliminate unpromising trials during the course of hyperparameter optimization.
pruners = {
    'BasePruner': optuna.pruners.BasePruner,                            # This is the base class for all pruning strategies in Optuna. It provides a skeleton for implementing custom pruning strategies.
    'MedianPruner': optuna.pruners.MedianPruner,                        # A pruner that prunes unpromising trials that have median objective values, as determined in previous steps.
    'SuccessiveHalvingPruner': optuna.pruners.SuccessiveHalvingPruner,  # This pruner repeatedly splits trials into halves, discarding the lower performing half at each iteration.
    'HyperbandPruner': optuna.pruners.HyperbandPruner,                  # This pruner implements the Hyperband algorithm, which selects promising trials and runs them with different resource allocation schemes to determine the best one.
    'PercentilePruner': optuna.pruners.PercentilePruner,                # A pruner that prunes unpromising trials based on their percentile rank relative to all completed trials.
    'NopPruner': optuna.pruners.NopPruner,                              # A pruner that does nothing and does not prune any trials.
    'ThresholdPruner': optuna.pruners.ThresholdPruner,                  # This pruner prunes trials that have not reached a certain level of performance (i.e., objective value).
    'PatientPruner': optuna.pruners.PatientPruner,                      # This pruner prunes trials that do not show improvement over a certain number of steps (or epochs).
}
halflives = {}
mol_features = {}

In [4]:
# load Jazzy features from csv files
for splitter in splitters:
    print("\n")
    print(splitter)
    mol_features[splitter] = {}
    halflives[splitter] = {}
    for isozyme in isozymes:
        mol_features[splitter][isozyme] = {}
        halflives[splitter][isozyme] = {}
        for split in data_splits:
            print(isozyme, split)
            df = pd.read_csv(rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"])
            features, thalfs, contains_nan = parse_jazzy_df(df)
            mol_features[splitter][isozyme][split] = features
            halflives[splitter][isozyme][split] = thalfs
            print(f"     {mol_features[splitter][isozyme][split][0]}")
            print(f"     {isozyme} mol_features {split} contains NaN: {contains_nan}")



rand
3A4 train
     56, [63, 'CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(Cl)cn1)[C@@]2(OCC1(O)CC1)c1ccc(Cl)cc1', 6.0, 13.1753, 1.4177, 11.3044, -7.3559, -125.8906, -119.4889]
     [  13.1753    1.4177   11.3044   -7.3559 -125.8906 -119.4889]
     3A4 mol_features train contains NaN: False
3A4 test
     14, [23, 'Cc1ncsc1-c1ccc([C@H](CC(=O)NCCCCCCNC(=O)COc2c(-c3csc(N4CCOCC4)n3)ccc(F)c2F)NC(=O)[C@@H]2C[C@@H](O)CN2C(=O)[C@@H](NC(=O)C2(F)CC2)C(C)(C)C)cc1', 0.3767, 28.4327, 3.274, 23.4872, -4.0348, -286.1545, -257.0326]
     [  28.4327    3.274    23.4872   -4.0348 -286.1545 -257.0326]
     3A4 mol_features test contains NaN: False
RLM train
     removed index 1047 corresponding to NaN
     removed index 1517 corresponding to NaN
     2022, [158, 'O=c1cc(N2CCOCC2)oc2c1ccc1ccccc12', 30.0, 8.0486, 0.0, 5.2165, -13.5913, -63.7029, -67.7504]
     [  8.0486   0.       5.2165 -13.5913 -63.7029 -67.7504]
     RLM mol_features train contains NaN: False
RLM test
     removed index 174 corresponding to Na

In [5]:
sampler = samplers['TPESampler']
pruner = pruners["BasePruner"]
n_trials = 200
for splitter in splitters:
    if splitter == "rand":
        splitter_name = "random"
    elif splitter == "scaff":
        splitter_name = "scaffold_splitter"
    else:
        splitter_name = "time_split"

    for isozyme in isozymes:
        X_train = mol_features[splitter][isozyme]["train"]
        y_train = np.log(halflives[splitter][isozyme]["train"])  # natural log half-lives -> values are less spread out
        X_test = mol_features[splitter][isozyme]["test"]
        y_test = np.log(halflives[splitter][isozyme]["test"])
        for model_identifier in model_identifiers:
            print(splitter_name, isozyme, model_identifier)
            study = optuna.create_study(study_name=model_identifier, directions=['minimize'], pruner=pruner,
                                        storage=f"sqlite:///project_resources/optuna/jazzy/{splitter_name}/{isozyme}/db.{model_identifier}", load_if_exists=True)
            test = HyperparamTuner(model_identifier, X_train, y_train, X_test, y_test)
            study.optimize(test.objective, n_trials=n_trials, n_jobs=-1)  # catch=(ValueError,)
            joblib.dump(study, f"./project_resources/optuna/jazzy/{splitter_name}/{isozyme}/{model_identifier}.pkl")

time_split 3A4 linear


[I 2023-10-14 14:03:11,387] A new study created in RDB with name: linear
[I 2023-10-14 14:03:12,116] Trial 6 finished with value: 1.387937501737305 and parameters: {'alpha': 0.07399118162364904, 'l1_ratio': 0.649867293705953}. Best is trial 6 with value: 1.387937501737305.
[I 2023-10-14 14:03:12,519] Trial 0 finished with value: 1.3821758903175214 and parameters: {'alpha': 0.0007336136174225366, 'l1_ratio': 0.28660384269281025}. Best is trial 0 with value: 1.3821758903175214.
[I 2023-10-14 14:03:12,568] Trial 3 finished with value: 1.4494818645292646 and parameters: {'alpha': 0.06702808075657941, 'l1_ratio': 0.6444685848397056}. Best is trial 0 with value: 1.3821758903175214.
[I 2023-10-14 14:03:12,743] Trial 10 finished with value: 1.4159700197782639 and parameters: {'alpha': 0.05865701916494759, 'l1_ratio': 0.28253216171000783}. Best is trial 0 with value: 1.3821758903175214.
[I 2023-10-14 14:03:12,826] Trial 7 finished with value: 1.3837471603560392 and parameters: {'alpha': 0.00810

time_split 3A4 KRR


[I 2023-10-14 14:03:39,789] Trial 4 finished with value: 1.4586872279578038 and parameters: {'alpha': 0.12659724279928897, 'gamma': 7.109940338278422e-15, 'kernel': 'laplacian'}. Best is trial 1 with value: 1.4227275009137184.
[I 2023-10-14 14:03:39,905] Trial 8 finished with value: 1.4411066440544962 and parameters: {'alpha': 0.40340695282777916, 'gamma': 8.899177178017992e-15, 'kernel': 'linear'}. Best is trial 3 with value: 1.4220809371480856.
[I 2023-10-14 14:03:39,918] Trial 0 finished with value: 1.4393353712470247 and parameters: {'alpha': 0.13128661703673924, 'gamma': 6.306016272458354e-15, 'kernel': 'laplacian'}. Best is trial 3 with value: 1.4220809371480856.
[I 2023-10-14 14:03:39,935] Trial 10 finished with value: 1.4926154655920907 and parameters: {'alpha': 0.5037599230564018, 'gamma': 6.842615852691461e-15, 'kernel': 'linear'}. Best is trial 3 with value: 1.4220809371480856.
[I 2023-10-14 14:03:40,031] Trial 1 finished with value: 1.4227275009137184 and parameters: {'alph

time_split 3A4 GB


...

[I 2023-10-14 15:17:37,199] A new study created in RDB with name: GB
[I 2023-10-14 15:17:43,301] Trial 2 finished with value: 0.625930778634336 and parameters: {'n_estimators': 50, 'learning_rate': 0.7359860684395833, 'max_depth': 1}. Best is trial 2 with value: 0.625930778634336.
[I 2023-10-14 15:17:43,593] Trial 11 finished with value: 0.6309333067260514 and parameters: {'n_estimators': 20, 'learning_rate': 0.6774476514128849, 'max_depth': 1}. Best is trial 2 with value: 0.625930778634336.
[I 2023-10-14 15:17:43,857] Trial 5 finished with value: 0.6310080909170441 and parameters: {'n_estimators': 10, 'learning_rate': 0.3030065585195304, 'max_depth': 3}. Best is trial 2 with value: 0.625930778634336.
[I 2023-10-14 15:17:44,154] Trial 10 finished with value: 0.610398042281657 and parameters: {'n_estimators': 50, 'learning_rate': 0.6975775083398761, 'max_depth': 1}. Best is trial 10 with value: 0.610398042281657.
[I 2023-10-14 15:17:44,200] Trial 9 finished with value: 0.638527373355364

time_split HLC RF


[I 2023-10-14 15:19:41,958] A new study created in RDB with name: RF
[I 2023-10-14 15:20:07,958] Trial 4 finished with value: 0.594440343112752 and parameters: {'n_estimators': 200, 'max_features': 'auto', 'max_depth': None}. Best is trial 4 with value: 0.594440343112752.
[I 2023-10-14 15:20:08,510] Trial 10 finished with value: 0.5906954713449402 and parameters: {'n_estimators': 200, 'max_features': 'auto', 'max_depth': None}. Best is trial 10 with value: 0.5906954713449402.
[I 2023-10-14 15:20:08,884] Trial 1 finished with value: 0.6006047093603383 and parameters: {'n_estimators': 20, 'max_features': 'sqrt', 'max_depth': 10}. Best is trial 10 with value: 0.5906954713449402.
[I 2023-10-14 15:20:09,144] Trial 5 finished with value: 0.58089415777365 and parameters: {'n_estimators': 20, 'max_features': 'auto', 'max_depth': 10}. Best is trial 5 with value: 0.58089415777365.
[I 2023-10-14 15:20:09,362] Trial 11 finished with value: 0.5989362922657236 and parameters: {'n_estimators': 20, 'm

time_split HLC ANN


[I 2023-10-14 15:27:15,205] A new study created in RDB with name: ANN
[I 2023-10-14 15:27:24,284] Trial 8 finished with value: 3.2777122466801956 and parameters: {'learning_rate_init': 0.07299673774166028, 'hidden_layer_sizes': [20, 20]}. Best is trial 8 with value: 3.2777122466801956.
[I 2023-10-14 15:27:24,787] Trial 9 finished with value: 2.6189718517591176 and parameters: {'learning_rate_init': 0.0761788534732596, 'hidden_layer_sizes': [50]}. Best is trial 9 with value: 2.6189718517591176.
[I 2023-10-14 15:27:25,197] Trial 1 finished with value: 2.6221887923939993 and parameters: {'learning_rate_init': 0.023521211992133074, 'hidden_layer_sizes': [10]}. Best is trial 9 with value: 2.6189718517591176.
[I 2023-10-14 15:27:25,530] Trial 10 finished with value: 2.630579643628572 and parameters: {'learning_rate_init': 0.07852137071109845, 'hidden_layer_sizes': [10, 10]}. Best is trial 9 with value: 2.6189718517591176.
[I 2023-10-14 15:27:27,394] Trial 11 finished with value: 2.7932304325